# Your Title Here

**Name(s)**: Kevin Wong, Andrew Yang

**Website Link**: (your website link)

# Introduction
<span style="color:red">Understand the data you have access to. Brainstorm a few questions that interest you about the dataset. Pick one question you plan to investigate further. (As the data science lifecycle tells us, this question may change as you work on your project.)</span>

# Univariate Analysis
<span style="color:red">Look at the distributions of relevant columns separately by using DataFrame operations and drawing at least two relevant plots</span>

# Bivariate Analysis
<span style="color:red">Look at the statistics of pairs of columns to identify possible associations. For instance, you may create scatter plots and plot conditional distributions, or box-plots. You must plot at least two such plots in your notebook. The results of your bivariate analyses will be helpful in identifying interesting hypothesis tests!	</span>

# Interesting Aggregates
<span style="color:red"> Choose columns to group and pivot by and examine aggregate statistics.	</span>

# NMAR Analysis
<span style="color:red"> Recall, to determine whether data are likely NMAR, you must reason about the data generating process; you cannot conclude that data are likely NMAR solely by looking at your data. As such, there’s no code to write here (and hence, nothing to put in your notebook).	</span>

# Missingness Dependency
<span style="color:red"> Pick a column in the dataset with non-trivial missingness to analyze, and perform permutation tests to analyze the dependency of the missingness of this column on other columns. </span>

<span style="color:red">Specifically, find at least one other column that the missingness of your selected column does depend on, and at least one other column that the missingness of your selected column does not depend on. </span>

<span style="color:red"> Tip: Make sure you know the difference between the different types of missingness before approaching that section. Many students in the past have lost credit for mistaking one type of missingness for another. </span>

# Hypothesis Testing
<span style="color:red"> Clearly state a pair of hypotheses and perform a hypothesis test or permutation test that is not related to missingness. Feel free to use the “sample questions” in each of the dataset descriptions or create your own. This should be the question that is stated clearly at the top of your report.	</span>

## Code

In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

In [ ]:
# TODO

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# TODO